In [166]:
import pandas as pd
from tqdm import tqdm
train_dp = "/opt/ml/dataset/train/train_revised.csv"
train_df = pd.read_csv(train_dp)

In [167]:
train_df

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...,...
32423,32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32424,32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32425,32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32426,32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [170]:
new_df = pd.DataFrame(
    [], columns=["Unnamed: 0","id", "sentence", "subject_entity", "object_entity", "label", "source"]
)
new_df

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source


In [171]:
change_sub_obj = ['no_relation', 'org:alternate_names','per:alternate_name','per:other_family','per:colleagues','per:siblings','per:spouse']
change_sub_obj_label = {'org:members':'org:members_of','org:members_of':'org:members',
                        'per:parents':'per:children','per:children':'per:parents'}

elsedata = {"org:top_members/employees": "per:employee_of"}


In [172]:
for i in tqdm(range(len(train_df))):
    if train_df['label'][i] in change_sub_obj:
        new_df.loc[len(new_df)] = [
        99,
        train_df.loc[i]["id"],
        train_df.loc[i]["sentence"],
        train_df.loc[i]["object_entity"], # 변경
        train_df.loc[i]["subject_entity"],
        train_df.loc[i]["label"],
        train_df.loc[i]["source"],
        ]
    elif train_df['label'][i] in change_sub_obj_label.keys():
        new_df.loc[len(new_df)] = [
        99,
        train_df.loc[i]["id"],
        train_df.loc[i]["sentence"],
        train_df.loc[i]["object_entity"], # 변경
        train_df.loc[i]["subject_entity"],
        change_sub_obj_label[train_df['label'][i]],
        train_df.loc[i]["source"],
        ]
    elif train_df['label'][i] in elsedata.keys():
        new_df.loc[len(new_df)] = [
        99,
        train_df.loc[i]["id"],
        train_df.loc[i]["sentence"],
        train_df.loc[i]["object_entity"], # 변경
        train_df.loc[i]["subject_entity"],
        elsedata[train_df['label'][i]],
        train_df.loc[i]["source"],
        ]

100%|██████████| 32428/32428 [01:30<00:00, 359.35it/s]


In [185]:
new_df

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
0,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
1,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
2,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
3,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
4,99,5,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...",org:alternate_names,wikipedia
...,...,...,...,...,...,...,...
18006,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
18007,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
18008,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
18009,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [158]:
new_df[new_df['label']=="per:employee_of"]

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
2,0,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
6,0,12,"2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《M...","{'word': '박찬호', 'start_idx': 39, 'end_idx': 41...","{'word': '필라델피아 필리스', 'start_idx': 19, 'end_id...",per:employee_of,wikipedia
21,0,36,곡성군(군수 유근기)은 상수도 사용 가구를 대상으로 올해부터 옥내 급수관 누수탐사 ...,"{'word': '유근기', 'start_idx': 7, 'end_idx': 9, ...","{'word': '곡성군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
23,0,39,앞서 구혜선은 안재현이 소속사인 HB엔터테인먼트의 문보미 대표와 자신을 험담했다고 ...,"{'word': '문보미', 'start_idx': 28, 'end_idx': 30...","{'word': 'HB엔터테인먼트', 'start_idx': 18, 'end_idx...",per:employee_of,wikitree
29,0,53,공무원연금공단(이사장 정남준)은 10월 28일부터 11월 1일까지 '인권 존중·배려...,"{'word': '정남준', 'start_idx': 12, 'end_idx': 14...","{'word': '공무원연금공단', 'start_idx': 0, 'end_idx':...",per:employee_of,wikitree
...,...,...,...,...,...,...,...
17986,0,32431,노무현 정부(참여 정부)는2002년 대통령 선거에서 새천년민주당 노무현 후보가 19...,"{'word': '노무현', 'start_idx': 36, 'end_idx': 38...","{'word': '참여 정부', 'start_idx': 7, 'end_idx': 1...",per:employee_of,wikipedia
17991,0,32439,서울 영등포구에 위치한 웨딩홀에서 진행된 이날 행사에는 김재겸 롯데홈쇼핑 지원본부장...,"{'word': '채현일', 'start_idx': 48, 'end_idx': 50...","{'word': '영등포구', 'start_idx': 64, 'end_idx': 6...",per:employee_of,wikitree
17999,0,32453,1952년 8월에 실시된 제2대 대통령 선거 후 집권당인 자유당의 내분과 함께 대한...,"{'word': '이승만', 'start_idx': 78, 'end_idx': 80...","{'word': '자유당', 'start_idx': 32, 'end_idx': 34...",per:employee_of,wikipedia
18008,0,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree


In [159]:
train_df[train_df['label']=="per:employee_of"]

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
9,9,9,"특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, 안규백 국회의원, 김종민 국회...","{'word': '안규백', 'start_idx': 30, 'end_idx': 32...","{'word': '더불어민주당', 'start_idx': 100, 'end_idx'...",per:employee_of,wikitree
15,15,15,"성명에는 더불어민주당 이상민 대전광역시당 위원장, 이춘희 세종특별자치시당 위원장, ...","{'word': '노영민', 'start_idx': 96, 'end_idx': 98...","{'word': '더불어민주당', 'start_idx': 5, 'end_idx': ...",per:employee_of,wikipedia
28,28,28,천도교 3대교주 손병희의 영향으로 태화관이 독립선언서 낭독장소가 되었다.,"{'word': '손병희', 'start_idx': 9, 'end_idx': 11,...","{'word': '천도교', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikipedia
34,34,34,이로써 아버지인 카를레스와 아들인 세르히오 부스케츠는 레알 마드리드의 마누엘 산치스...,"{'word': '마누엘 산치스', 'start_idx': 39, 'end_idx'...","{'word': '레알 마드리드', 'start_idx': 30, 'end_idx'...",per:employee_of,wikipedia
74,74,74,"일부는 미국 독립 전쟁 중에 활약을 하였으며, 이던 앨런이 이끄는 그린 마운틴 보이...","{'word': '이던 앨런', 'start_idx': 26, 'end_idx': ...","{'word': '그린 마운틴 보이스', 'start_idx': 37, 'end_i...",per:employee_of,wikipedia
...,...,...,...,...,...,...,...
32399,32441,32441,"윤종필 자유한국당 의원은 이날 ""현재 청소년들 휴대폰 과의존 문제가 심각하다""라며 ...","{'word': '윤종필', 'start_idx': 0, 'end_idx': 2, ...","{'word': '자유한국당', 'start_idx': 4, 'end_idx': 8...",per:employee_of,wikitree
32400,32442,32442,"해병대의 원산 상륙으로 놀란 이후, 중화인민공화국 주석 마오쩌둥은 전보를 통해 중국...","{'word': '마오쩌둥', 'start_idx': 31, 'end_idx': 3...","{'word': '중화인민공화국', 'start_idx': 20, 'end_idx'...",per:employee_of,wikipedia
32407,32449,32449,"이때 만주 지역에는 크고작은 독립군이 조직되었고, 동만주지방의 홍범도가 이끄는 대한...","{'word': '홍범도', 'start_idx': 35, 'end_idx': 37...","{'word': '대한독립군', 'start_idx': 44, 'end_idx': ...",per:employee_of,wikipedia
32419,32461,32461,1852년 미국 대통령 선거에서 휘그당은 미국-멕시코 전쟁의 영웅 테일러 장군의 사...,"{'word': '밀러드 필모어', 'start_idx': 66, 'end_idx'...","{'word': '휘그당', 'start_idx': 18, 'end_idx': 20...",per:employee_of,wikipedia


In [100]:
# 문장 중복 확인
for i in tqdm(range(len(train_df))):
    #print('\n\n')
    #print('org: ',train_df['sentence'][i],'\n',train_df['subject_entity'][i],'\n',train_df['object_entity'][i],'\n',train_df['label'][i])
    for j in range(len(train_df)):
        if i == j:
            pass
        else:
            if train_df['sentence'][i] == train_df['sentence'][j]:
                #print('same: ',train_df['sentence'][j],'\n',train_df['subject_entity'][j],'\n',train_df['object_entity'][j],'\n',train_df['label'][j])
                if train_df['subject_entity'][i] == train_df['subject_entity'][j] and train_df['object_entity'][i] == train_df['object_entity'][j] and train_df['label'][i] == train_df['label'][j]:
                    print('exactly same')

  0%|          | 48/32428 [00:16<3:06:58,  2.89it/s]


KeyboardInterrupt: 

In [174]:
aug_df = train_df.append(new_df, ignore_index=True)

In [175]:
aug_df

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [176]:
aug_df[aug_df['Unnamed: 0']==99]

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [177]:
only_aug = aug_df[aug_df['Unnamed: 0']==99]

In [178]:
aug_df_drop = only_aug.drop_duplicates(subset=['subject_entity', 'object_entity', 'sentence','label'], keep='first')

In [179]:
aug_df_drop

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [164]:
aug_df_drop = aug_df_drop.drop(['Unnamed: 0'],axis=1)

In [180]:
aug_df_drop

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [181]:
aug_df_drop.to_csv('./only_aug.csv')

In [184]:
dd = "/opt/ml//only_aug.csv"
dd = pd.read_csv(dd)
dd

,Unnamed: 0,Unnamed: 0.1,id,sentence,subject_entity,object_entity,label,source
0,99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia
1,32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
2,32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
3,32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
4,32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...,...,...
18007,50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
18008,50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
18009,50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
18010,50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [187]:
only_aug

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [197]:
import numpy as np
# no relation/per/org 나누기
type_sep = [
    (only_aug['label']=='no_relation'),
    (only_aug['label'].str.startswith('p')),
    (only_aug['label'].str.startswith('o'))
]
type_sep_list = ['no_relation', 'per','org']

only_aug['type'] = np.select(type_sep, type_sep_list,default = None)

<ipython-input-197-425da285c98b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_aug['type'] = np.select(type_sep, type_sep_list,default = None)


In [198]:
# sub_entity type
type_sep = [
    (only_aug['subject_entity'].str.endswith('G\'}')),
    (only_aug['subject_entity'].str.endswith('R\'}'))
]  
type_sep_list = ['org', 'per']

only_aug['sub_type'] = np.select(type_sep, type_sep_list,default = None)

# object_entity type
type_sep = [
    (only_aug['object_entity'].str.endswith('PER\'}')),
    (only_aug['object_entity'].str.endswith('LOC\'}')),
    (only_aug['object_entity'].str.endswith('POH\'}')),
    (only_aug['object_entity'].str.endswith('DAT\'}')),
    (only_aug['object_entity'].str.endswith('NOH\'}')),
    (only_aug['object_entity'].str.endswith('ORG\'}'))
]  
type_sep_list = ['per','loc','poh','dat','noh','org']

only_aug['object_type'] = np.select(type_sep, type_sep_list,default = None)

only_aug['entity'] = only_aug['sub_type'] +':'+ only_aug['object_type']

<ipython-input-198-a13d0af13768>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_aug['sub_type'] = np.select(type_sep, type_sep_list,default = None)
<ipython-input-198-a13d0af13768>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_aug['object_type'] = np.select(type_sep, type_sep_list,default = None)
<ipython-input-198-a13d0af13768>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [202]:
only_aug[only_aug['object_type']=='org']

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source,type,sub_type,object_type,entity
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia,org,org,org,org:org
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia,no_relation,per,org,per:org
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree,no_relation,org,org,org:org
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree,per,per,org,per:org
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia,no_relation,None,org,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50431,99,32459,"메이저 리그 10승 경력을 높이 사서, 2011년 12월 2일 삼성 라이온즈에 영입...","{'word': '2011년', 'start_idx': 22, 'end_idx': ...","{'word': '삼성 라이온즈', 'start_idx': 35, 'end_idx'...",no_relation,wikipedia,no_relation,None,org,NaN
50432,99,32460,위광환 전라남도 해양수산국장은 “지난 3차례 스마트양식 클러스터 조성 공모를 통해 ...,"{'word': '전라남도', 'start_idx': 4, 'end_idx': 7,...","{'word': '영광군', 'start_idx': 72, 'end_idx': 74...",no_relation,wikitree,no_relation,None,org,NaN
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree,per,per,org,per:org
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia,no_relation,org,org,org:org


In [209]:
only_aug['object_type'].summary()

AttributeError: 'Series' object has no attribute 'summary'

In [199]:
only_aug

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source,type,sub_type,object_type,entity
99,99,99,2007년 12월 3일 울산방송(ubc)이 특별기획으로 마련한 '대통령후보에게 듣는...,"{'word': '울산방송', 'start_idx': 13, 'end_idx': 1...","{'word': 'ubc', 'start_idx': 18, 'end_idx': 20...",org:alternate_names,wikipedia,org,org,org,org:org
32428,99,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia,no_relation,per,org,per:org
32429,99,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree,no_relation,org,org,org:org
32430,99,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",per:employee_of,wikitree,per,per,org,per:org
32431,99,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia,no_relation,None,org,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50434,99,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...",per:other_family,wikipedia,per,per,per,per:per
50435,99,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia,per,per,per,per:per
50436,99,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree,per,per,org,per:org
50437,99,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia,no_relation,org,org,org:org
